**Import required libraries and scripts**

In [1]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.consensus_methods import *
from scripts.performance_calculation import *
from scripts.dogsitescorer import *
from scripts.get_pocket import *
from scripts.postprocessing import *
from scripts.protein_preparation import *

[21:49:04] Initializing Normalizer
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/MDAnalysis/coordinates/chemfiles.py:108: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_CHEMFILES_VERSION = LooseVersion("0.9")
[TRJ.py:171 - <module>()] netCDF4 is not available. Writing AMBER ncdf files will be slow.
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/MDAnalysis/coordinates/TRJ.py:1209: DeprecationWarning: Please use `netcdf_file` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  class NCDFPicklable(scipy.io.netcdf.netcdf_file):
/home/tony/.conda/envs/dockm8/lib/python3.10/site-packages/matplotlib/__init__.py:152: DeprecationWar

In [2]:
software = Path('/home/tony/DockM8/software')
receptor = Path('/home/tony/DockM8/testing_single_docking/protein.pdb')
ref_file = Path('/home/tony/DockM8/testing_single_docking/ref.sdf')
docking_library = Path('/home/tony/DockM8/testing_single_docking/library.sdf')
docking_programs = ['GNINA', 'SMINA', 'PLANTS', 'QVINAW', 'QVINA2']
clustering_metrics = ['RMSD', 'spyRMSD', 'espsim', '3DScore', 'bestpose', 'bestpose_GNINA', 'bestpose_SMINA', 'bestpose_PLANTS', 'bestpose_QVINAW', 'bestpose_QVINA2']
rescoring_functions = ['GNINA_Affinity','CNN-Score','CNN-Affinity', 'AD4', 'CHEMPLP', 'RFScoreVS', 'LinF9', 'SCORCH', 'Vinardo', 'PLECScore', 'NNScore', 'KORPL', 'ConvexPLR', 'AAScore']
id_column = 'ID'
n_poses = 10
exhaustiveness = 8
protonation = 'GypsumDL'
ncpus = 5
pocket = 'reference'
prepare_protein = 1
open('log.txt', 'w').close()


In [3]:
if prepare_protein == 1:
    #Prepare the protein
    prepared_receptor = prepare_protein_protoss(receptor)
else:
    prepared_receptor = receptor

#Create a temporary folder for all further calculations
w_dir = prepared_receptor.parent / prepared_receptor.stem
print('The working directory has been set to:', w_dir)
(w_dir).mkdir(exist_ok=True)

if pocket == 'reference':
    pocket_definition = get_pocket(ref_file, prepared_receptor, 8)
    print(pocket_definition)
if pocket == 'RoG':
    pocket_definition = get_pocket_RoG(ref_file, prepared_receptor)
    print(pocket_definition)
elif pocket == 'dogsitescorer':
    pocket_definition = binding_site_coordinates_dogsitescorer(prepared_receptor, w_dir, method='volume')
    print(pocket_definition)


[2023-Nov-20 21:49:12]: Preparing protein with ProtoSS ...
Job 2f7717e1-2782-41dc-af34-a32d7a9f1099 completed with success
The working directory has been set to: /home/tony/DockM8/testing_single_docking/protein_protoss

[2023-Nov-20 21:49:14]: Extracting pocket from protein_protoss using ref as reference ligand

[2023-Nov-20 21:49:59]: Finished extracting pocket from protein_protoss using ref as reference ligand
{'center': [16.7, -2.69, 17.61], 'size': [16.0, 16.0, 16.0]}


In [ ]:
if (w_dir / 'final_library.sdf').is_file() == False:
    prepare_library(docking_library, w_dir, id_column, protonation, software, ncpus)

In [4]:
docking(w_dir, prepared_receptor, pocket_definition, software, 'PLANTS', exhaustiveness, n_poses, 5, 'multiprocessing')
concat_all_poses(w_dir, docking_programs, prepared_receptor)



[2023-Nov-20 21:50:00]: Split final library folder already exists...


Fetching PLANTS docking poses: 100%|██████████| 1/1 [00:00<00:00, 929.79it/s]


[2023-Nov-20 21:50:00]: ERROR: Failed to write combined PLANTS docking poses

[2023-Nov-20 21:50:00]: No objects to concatenate

[2023-Nov-20 21:50:00]: Docking split files using SMINA...



100%|██████████| 10/10 [05:07<00:00, 30.75s/it]



[2023-Nov-20 21:55:07]: Docking with SMINA complete in 307.7900!


...

Converting molecules to .pdbqt using Meeko
Converting molecules to .pdbqt using MeekoConverting molecules to .pdbqt using MeekoConverting molecules to .pdbqt using Meeko


Converting molecules to .pdbqt using Meeko
Converting molecules to .pdbqt using Meeko
Converting molecules to .pdbqt using MeekoConverting molecules to .pdbqt using Meeko
Converting molecules to .pdbqt using Meeko

Converting molecules to .pdbqt using Meeko


100%|██████████| 10/10 [00:00<00:00, 109.26it/s]



[2023-Nov-20 21:59:42]: Docking with QVINA2 complete in 0.4188!


Loading QVINA2 poses: 0it [00:00, ?it/s]
[posebusters.py:55 - __init__()] Using configuration dictionary provided by user.



[2023-Nov-20 21:59:42]: ERROR: Failed to Load QVINA2 poses SDF file!

[2023-Nov-20 21:59:42]: No objects to concatenate

[2023-Nov-20 21:59:42]: ERROR: Failed to write combined QVINA2 poses SDF file!

[2023-Nov-20 21:59:42]: local variable 'qvina2_df' referenced before assignment

[2023-Nov-20 21:59:42]: ERROR: Failed to load PLANTS SDF file!

[2023-Nov-20 21:59:42]: [Errno 2] No such file or directory: '/home/tony/DockM8/testing_single_docking/protein_protoss/plants/plants_poses.sdf'

[2023-Nov-20 21:59:42]: ERROR: Failed to load QVINA2 SDF file!

[2023-Nov-20 21:59:42]: [Errno 2] No such file or directory: '/home/tony/DockM8/testing_single_docking/protein_protoss/qvina2/qvina2_poses.sdf'

[2023-Nov-20 22:00:36]: PoseBusters checking completed in 53.25 seconds.

[2023-Nov-20 22:00:36]: All poses succesfully checked and combined!


In [ ]:
print('Loading all poses SDF file...')
tic = time.perf_counter()
all_poses = PandasTools.LoadSDF(str(w_dir / 'allposes.sdf'), idName='Pose ID', molColName='Molecule', includeFingerprints=False, strictParsing=True)
print(f'Loaded {len(all_poses)} poses.')
toc = time.perf_counter()
print(f'Finished loading all poses SDF in {toc-tic:0.4f}!...')

for metric in clustering_metrics:
    if os.path.isfile(w_dir / f'clustering/{metric}_clustered.sdf') == False:
        filtered_poses = cluster_pebble(metric, 'KMedoids', w_dir, prepared_receptor, pocket_definition, software, all_poses, ncpus)


In [ ]:
for metric in clustering_metrics:
        rescore_poses(w_dir, prepared_receptor, pocket_definition, software, w_dir / 'clustering' / f'{metric}_clustered.sdf', rescoring_functions , ncpus)


In [ ]:
apply_consensus_methods(w_dir, 'bestpose', 'method6', rescoring_functions, 'min_max')